In [1]:
import ee
ee.Initialize()

In [2]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

Folium version: 0.10.0


In [3]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = EE_TILES.format(**v),
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [4]:
const1 = ee.Image(1)
const2 = ee.Image(2)

collection = ee.ImageCollection([const1, const2])
print(collection)
print(collection.getInfo())

ee.ImageCollection({
  "type": "Invocation",
  "arguments": {
    "images": [
      {
        "type": "Invocation",
        "arguments": {
          "value": 1
        },
        "functionName": "Image.constant"
      },
      {
        "type": "Invocation",
        "arguments": {
          "value": 2
        },
        "functionName": "Image.constant"
      }
    ]
  },
  "functionName": "ImageCollection.fromImages"
})
{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}], 'properties': {'system:index': '0'}}, {'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}], 'properties': {'system:index': '1'}}]}


In [7]:
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
               .filter(ee.Filter.eq('WRS_PATH', 44))\
               .filter(ee.Filter.eq('WRS_ROW', 34))\
               .filterDate('2014-03-01', '2014-08-01')
            
collection.getInfo()

{'type': 'ImageCollection',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B7', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B8', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B9', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B10', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B11', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'BQA',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535}}],
 'id': 'LANDSAT/LC08/C01/T1_TOA

In [9]:
count = collection.size()
count.getInfo()

9

In [11]:
from datetime import datetime as dt

rango = collection.reduceColumns(ee.Reducer.minMax(), ['system:time_start'])

init_date = ee.Date(rango.get('min')).getInfo()['value']/1000
init_date_f = dt.utcfromtimestamp(init_date).strftime('%Y-%m-%d %H:%M:%S')

last_date = ee.Date(rango.get('max')).getInfo()['value']/1000
last_date_f = dt.utcfromtimestamp(last_date).strftime('%Y-%m-%d %H:%M:%S')

print('Date range:', init_date_f, last_date_f)

Date range: 2014-03-18 18:46:32 2014-07-24 18:45:57


In [20]:
good_qty = collection.sort('CLOUD_COVER').limit(5)
good_qty.getInfo()

{'type': 'ImageCollection',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B7', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B8', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B9', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B10', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'B11', 'data_type': {'type': 'PixelType', 'precision': 'float'}},
  {'id': 'BQA',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535}}],
 'id': 'LANDSAT/LC08/C01/T1_TOA

In [21]:
collection = ee.ImageCollection('LANDSAT/LT05/C01/T2').filterDate('1987-01-01', '1990-05-01').filterBounds(ee.Geometry.Point(25.8544, -18.08874))

filtered = collection.filterMetadata('IMAGE_QUALITY', 'equals', 9)

# Create two composites to check the effect of filtering by IMAGE_QUALITY.
badComposite = ee.Algorithms.Landsat.simpleComposite(collection, 75, 3)
goodComposite = ee.Algorithms.Landsat.simpleComposite(filtered, 75, 3)

dicc = {
    'Bad composite' : badComposite.getMapId({'bands': ['B3', 'B2', 'B1'], 'gain': 3.5}),
    'Good composite': goodComposite.getMapId({'bands': ['B3', 'B2', 'B1'], 'gain': 3.5})
}


# Display the results
center = [-18.08874, 25.8544]
Mapdisplay(center, dicc, zoom_start= 13)

In [24]:
# Load a Landsat 8 collection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filter(ee.Filter.eq('WRS_PATH', 44)).filter(ee.Filter.eq('WRS_ROW', 34)).filterDate('2014-01-01', '2015-01-01')

# Compute a median image and display.
median = collection.median()

dicc = {
    'median' : median.getMapId({'bands': ['B4', 'B3', 'B2'], 'max': 0.3})
}

# Display the results
center = [37.7726, -122.3578]
Mapdisplay(center, dicc, zoom_start= 12)